In [ ]:
# to deploy an app
# !pip install fastapi
# !pip install uvicorn
# !pip install sse-starlette

#!/usr/bin/env python
import sys
import os

# Add the parent directory (your_project) to sys.path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from fastapi import FastAPI
import uvicorn

from langserve import add_routes
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

from utils.inmemoryhistory import get_by_session_id
from config_input import InputChat


# Define the model
model = Ollama(model='llama2')

# fills the templates
system_template = """You are Professor Oak, a world-renowned Pokémon Professor from Pallet Town. Your expertise lies exclusively in Pokémon, 
and you have very limited knowledge of real-world animals.

Instructions:
1. If you know the answer, respond confidently and clearly.
2. Keep your answers short and to the point and avoid referring back to earlier discussions.
3. You dont know about real animals—only Pokémon.
4. Whenever someone mentions an animal, you will assume they are referring to a Pokémon that closely resembles that animal. 
5. You will describe the Pokémon in detail, including its type, abilities, habitat, and any unique traits it has, as if it is the animal in question. 
6. You should always try to connect it back to your vast knowledge of Pokémon.

For example:

If someone mentions a "lion," you might think they are talking about Luxray or Pyroar.
If someone talks about a "turtle," you might believe they are referring to Blastoise or Torkoal.
"""
human_template = "{human_input}"
history_template = [
    ("human", "Professor Oak, I want to be Pokémon Trainer and catch 'em all!"), 
    ("ai", """Ah, so you want to become a Pokémon Trainer, do you? That's quite the ambitious goal!
    First things first, you will need a partner. Have you thought about which starter Pokémon you want to choose?"""),
]

prompt_history  = MessagesPlaceholder(variable_name="history", optional=True)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    history_template[0], # you keep the history template for all the sessions_id to refort the prompt
    prompt_history,
    ("human", human_template),
])

# NOTE: The code above is if you want to keep the history to a specifict chat. But you can keep the chat for all sessions_id
# initialize the session_id that you want
# history = get_by_session_id(session_id)

# now save to the model the conversation in the same session_id that you want
# history.add_messages(history_template)

# start the chain
chain = prompt_template | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="human_input",
    history_messages_key="history",
)


# App definition
app = FastAPI(
  title="LangChain Oak Server",
  version="0.1",
  description="A simple API server using LangChain's Runnable interfaces",
)

# Adding chain route
add_routes(
    app,
    chain_with_history.with_types(input_type=InputChat, output_type=str),
    path="/oak",
    # playground_type="chat",
    # enable_feedback_endpoint=True,
    # enable_public_trace_link_endpoint=True,
)


if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)


In [ ]:
# %%
#!/usr/bin/env python
import uvicorn
from fastapi import FastAPI

from langserve import add_routes
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# Import custom tools
from tools.get_evolution import get_evolution
from tools.random_movements import random_movements
from tools.get_pokedex_data import get_pokedex_data

# Import the custom type of the input
from config.config_input import InputChat
from config.config_stategraph import AgentState
from config.config_fewshot import history_template
from config.config_prompting import system_template

# %%
## Model Configuration

# Initialize ChatOllama model
model = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

# %%
## Prompt Templates
# # This is a prompt template used to format each individual example.
# few_shot_map = ChatPromptTemplate.from_messages(
#     [
#         ("human", "{input}"),
#         ("ai", "{output}"),
#     ]
# )
# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=few_shot_map,
#     examples=few_shot_template,
# )

# Create prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    # few_shot_prompt,
    history_template[0],
    MessagesPlaceholder(variable_name="messages"),
])

# %%
# Define tools
tools = [random_movements, get_pokedex_data, get_evolution]

# Bind tools to the model
model_with_tools = model.bind_tools(tools)

# Create the chain
chain = prompt_template | model_with_tools

# %%
## Graph Functions

def should_continue(state):
    """Determine whether to continue or end the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    return "continue" if last_message.tool_calls else "end"

def call_model_chained(state):
    messages = state["messages"]
    response = chain.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# %%
## Graph Configuration

# Define a new graph
workflow = StateGraph(AgentState)

# Define the function to execute tools
tool_node = ToolNode(tools)

# Define the checkpointer
memory = MemorySaver()

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model_chained)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
oak_graph = workflow.compile(checkpointer=memory)

## FastAPI App Configuration
app = FastAPI(
    title="LangChain Oak Graph Server",
    version="0.1",
    description="A simple API server using LangChain's Runnable interfaces",
)

# Add routes
add_routes(
    app,
    oak_graph.with_types(input_type=InputChat, output_type=dict), 
    path="/oak",
)

## Main Execution
if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)

In [ ]:
from langchain_core.tools import ToolException
from langchain_core.tools import tool
import requests

@tool
def get_evolution(pokemon_name: str) -> list:
    """This is a method to give you a information of the evolution path of a certain pokemon

    Args:
        pokemon_name: a pokemon name given by the user.
    """

    print(f"Calling the tool get_evolution to {pokemon_name.lower()}")

    species_url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_name.lower()}"
    species_response = requests.get(species_url)

    if species_response.status_code != 200:
        raise ToolException(f"Error: {pokemon_name} is not a valid pokemon")
    
    species_data = species_response.json()

    # Step 2: Extract evolution chain URL from species data
    evolution_chain_url = species_data['evolution_chain']['url']

    # Step 3: Get the evolution chain data
    evolution_response = requests.get(evolution_chain_url)
    evolution_data = evolution_response.json()

    # Step 4: Traverse the evolution chain and get the names of evolutions
    evolutions = []
    current_evolution = evolution_data['chain']
    
    while current_evolution:
        evolutions.append(current_evolution['species']['name'])
        if len(current_evolution['evolves_to']) > 0:
            current_evolution = current_evolution['evolves_to'][0]
        else:
            break
    
    return evolutions

In [ ]:
from langchain_core.tools import ToolException
from langchain_core.tools import tool
import requests
import random
from typing import List

@tool
def random_movements(pokemon_name: str) -> List[str]:
    """This is a method to give you a random movements list of a certain pokemon if the user asks for them

    Args:
        pokemon_name: a pokemon name given by the user.
    """

    print(f"Calling the tool random_movements to {pokemon_name.lower()}")

    # The url of the api
    url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name.lower()}'
        
    # Make the API request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        raise ToolException(f"Error: {pokemon_name} is not a valid pokemon")

    # Parse the response JSON
    data = response.json()

    # Extract the list of moves using map and lambda
    moves = list(map(lambda move: move['move']['name'], data['moves']))

    if len(moves) < 4:
        return moves

    # Select 4 random
    selected_moves = random.sample(moves, 4)

    return selected_moves